In [101]:
import pandas as pd
import numpy as np

In [158]:
import importlib
import models
import utils
importlib.reload(models)
importlib.reload(utils)
from models import get_prediction
import torch
import librosa
import joblib
from IPython.display import Audio as display_audio

In [5]:
DESED_DATA = '/Volumes/DataSSD/SoundSegment/dataset/'
STRONG_META = DESED_DATA + 'metadata/validation/'
STRONG_AUDIO = DESED_DATA + 'audio/validation/validation/'

In [96]:
fold_models = [torch.load(f'trained_models/model_fold_{fold}.pth') for fold in range(4)]
label_encoder = joblib.load('label_encoder.pkl')

In [6]:
df = pd.read_csv(STRONG_META + 'validation.tsv', sep='\t')
df.head(10)

,filename,onset,offset,event_label
0,Y00pbt6aJV8Y_350.000_360.000.wav,0.000,9.971,Vacuum_cleaner
1,Y00pK0GMmE9s_70.000_80.000.wav,0.000,10.000,Vacuum_cleaner
2,Y02sD1KJeoGA_50.000_60.000.wav,0.000,10.000,Frying
3,Y0bjUq9XMMmQ_30.000_40.000.wav,0.000,10.000,Frying
4,Y0cH_NlhhMAs_30.000_40.000.wav,1.710,6.005,Cat
5,Y0eh_N-cmcuI_350.000_360.000.wav,0.467,0.717,Alarm_bell_ringing
6,Y0eh_N-cmcuI_350.000_360.000.wav,1.919,10.000,Running_water
7,Y0eh_N-cmcuI_350.000_360.000.wav,5.418,5.863,Speech
8,Y0eh_N-cmcuI_350.000_360.000.wav,7.790,10.000,Speech
9,Y0hAK5f1tDok_30.000_40.000.wav,6.367,7.226,Cat


In [7]:
df = df[df['event_label'].isin(['Dog', 'Cat', 'Vacuum_cleaner', 'Running_water'])]

In [8]:
df

,filename,onset,offset,event_label
0,Y00pbt6aJV8Y_350.000_360.000.wav,0.000,9.971,Vacuum_cleaner
1,Y00pK0GMmE9s_70.000_80.000.wav,0.000,10.000,Vacuum_cleaner
4,Y0cH_NlhhMAs_30.000_40.000.wav,1.710,6.005,Cat
6,Y0eh_N-cmcuI_350.000_360.000.wav,1.919,10.000,Running_water
9,Y0hAK5f1tDok_30.000_40.000.wav,6.367,7.226,Cat
...,...,...,...,...
4077,Yc1_V2y_9KCM_369.000_379.000.wav,7.611,8.604,Running_water
4161,YZfDu5aHVrAg_205.000_215.000.wav,0.000,1.888,Running_water
4163,YZfDu5aHVrAg_205.000_215.000.wav,2.090,5.618,Running_water
4201,Y5iARe0XPQdg_95.000_105.000.wav,0.000,1.021,Running_water


In [10]:
x, _ = librosa.load(STRONG_AUDIO + df.loc[0, 'filename'])

In [27]:
cls_map = dict([(0, 'dog'), (5, 'cat'), (17, 'pouring_water'), (36, 'vacuum_cleaner')])

In [46]:
frame_df, clip = get_prediction(model, torch.tensor(x), label_encoder, cls_map)
clip

'dog'

In [55]:
desed_map = {'dog': 'Dog', 'cat': 'Cat', 'vacuum_cleaner': 'Vacuum_cleaner', 'pouring_water': 'Running_water'}

In [57]:
frame_df['event_label'] = frame_df['event'].map(desed_map)

In [58]:
frame_df[['onset', 'offset', 'event_label']] .to_csv('fp.csv', header=False)

In [31]:
display_audio(STRONG_AUDIO + df.loc[0, 'filename'])

In [155]:
audio, _ = librosa.load(STRONG_AUDIO + df.loc[0, 'filename'])
audio = torch.tensor(audio).unsqueeze(0)
pp(fold_models, audio)

(tensor([[0.5201, 0.6298, 0.4032, 0.5416],
         [0.5201, 0.6298, 0.4032, 0.5416],
         [0.5201, 0.6298, 0.4032, 0.5416],
         ...,
         [0.4285, 0.4405, 0.4473, 0.4953],
         [0.4285, 0.4405, 0.4473, 0.4953],
         [0.4285, 0.4405, 0.4473, 0.4953]], grad_fn=<DivBackward0>),
 1)

In [160]:
import os
os.makedirs('evals', exist_ok=True)

file_list = []
for idx, (index, group) in enumerate(df.groupby('filename')):
    try:
        audio, _ = librosa.load(STRONG_AUDIO + index)
        pred_df, _ = get_prediction(fold_models, torch.tensor(audio), label_encoder, cls_map)
        pred_df['event_label'] = pred_df['event'].map(desed_map)
        group[['onset', 'offset', 'event_label']].to_csv(f'evals/{index}.txt', header=False, index=False)
        pred_df[['onset', 'offset', 'event_label']] .to_csv(f'evals/{index}_preds.txt', header=False, index=False)

        file_list.append(f'evals/{index}.txt  evals/{index}_preds.txt')
    except Exception as e:
#         print(e)
        print(f'Error for file {index}')
        pass
    

/opt/miniconda3/envs/stand/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error for file Y-9byM5-ih2M_0.000_10.000.wav


/opt/miniconda3/envs/stand/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error for file Y0R4uO4Vy9c0_110.000_120.000.wav


/opt/miniconda3/envs/stand/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


...

Error for file YqMDpBAs38EA_10.000_20.000.wav


/opt/miniconda3/envs/stand/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error for file YrhhoC11SBi4_0.000_10.000.wav


/opt/miniconda3/envs/stand/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error for file YskSyu-Q5nZI_0.000_10.000.wav


In [161]:
len(file_list)

493

In [162]:
with open('file_list.txt', 'w') as f:
    for line in file_list:
        f.write(';'.join(line.replace('  ', ' ').split(' ')) + '\n')

In [163]:
!./sed_eval-master/evaluators/sound_event_eval.py file_list.txt

Segment based metrics
  Evaluated length                  : 4262.26 sec
  Evaluated files                   : 493 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 14.99 %
    Precision                       : 16.70 %
    Recall                          : 13.60 %
  Error rate
    Error rate (ER)                 : 1.37 
    Substitution rate               : 0.17 
    Deletion rate                   : 0.69 
    Insertion rate                  : 0.51 
  Accuracy
    Sensitivity                     : 13.60 %
    Specificity                     : 82.41 %
    Balanced accuracy               : 48.01 %
    Accuracy                        : 68.24 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 14.96 %
    Precision                       : 18.21 %
    Recall                          : 14.20 %
  Error rate
    Error rate (ER)                 : 1.62 
    Deletion rat